# Learn the RNN Network

## First, learn the basics of RNNs

In [7]:
import numpy as np

# تنظیمات اولیه
input_size = 1
hidden_size = 4
output_size = 1
# طول دنباله زمانی یا رابطه ای که در اینجا جمله سه کلمه ای است
seq_length = 3

# وزن‌ها و بایاس‌ها
W_x = np.random.randn(hidden_size, input_size)
W_h = np.random.randn(hidden_size, hidden_size)
W_y = np.random.randn(output_size, hidden_size)
b_h = np.zeros((hidden_size, 1))
b_y = np.zeros((output_size, 1))

print(f"x Weight matrix:\n{W_x}\n")
print(f"Hidden Weight matrix:\n{W_h}\n")
print(f"output(y) Weight matrix:\n{W_y}\n")
print(f"Hidden bias:\n{b_h}\n")
print(f"Output bias:\n{b_y}\n")


# تابع فعال‌سازی
def tanh(x):
    return np.tanh(x)

# ورودی ساده
inputs = [np.array([[0.1]]), np.array([[0.2]]), np.array([[0.3]])]
print(f"input: {inputs}")

# حالت پنهان اولیه
h_prev = np.zeros((hidden_size, 1))

# اجرای RNN
for t in range(seq_length):
    x_t = inputs[t]
    h_t = tanh(np.dot(W_x, x_t) + np.dot(W_h, h_prev) + b_h)
    y_t = np.dot(W_y, h_t) + b_y
    print(f"Time {t}: Output = {y_t.ravel()}")
    h_prev = h_t


x Weight matrix:
[[ 0.24499315]
 [ 1.64245811]
 [-0.46516492]
 [ 0.60331829]]

Hidden Weight matrix:
[[ 1.37777454 -1.41781434  0.89708677  0.12688328]
 [-0.59478309 -0.19901797 -0.36983813  1.8497388 ]
 [ 0.00851169 -1.09938783  2.00037938 -1.15957504]
 [-1.73456868  0.01300878  0.9359561   1.36862273]]

output(y) Weight matrix:
[[ 1.73178273 -2.14228566 -0.53759098  0.17637289]]

Hidden bias:
[[0.]
 [0.]
 [0.]
 [0.]]

Output bias:
[[0.]]

input: [array([[0.1]]), array([[0.2]]), array([[0.3]])]
Time 0: Output = [-0.27069536]
Time 1: Output = [-0.90355753]
Time 2: Output = [-2.36305458]


## Make a model with learning the weights

In [28]:
import numpy as np

# تنظیمات
input_size = 1
hidden_size = 4
output_size = 1
seq_length = 3
learning_rate = 0.01

# وزن‌ها
W_x = np.random.randn(hidden_size, input_size)
W_h = np.random.randn(hidden_size, hidden_size)
W_y = np.random.randn(output_size, hidden_size)
b_h = np.zeros((hidden_size, 1))
b_y = np.zeros((output_size, 1))


# تابع فعال‌سازی و مشتقش
def tanh(x):
    return np.tanh(x)


def dtanh(x):
    return 1 - np.tanh(x) ** 2


# داده‌ها
inputs = [np.array([[0.1]]), np.array([[0.2]]), np.array([[0.3]])]
targets = [np.array([[0.2]]), np.array([[0.3]]), np.array([[0.4]])]

# آموزش
for epoch in range(1000):
    # hidden states
    h_states = []
    outputs = []
    # previous hidden state
    h_prev = np.zeros((hidden_size, 1))

    # Forward pass
    for x_t in inputs:
        h_t = tanh(np.dot(W_x, x_t) + np.dot(W_h, h_prev) + b_h)
        y_t = np.dot(W_y, h_t) + b_y
        h_states.append(h_t)
        outputs.append(y_t)
        h_prev = h_t

    # محاسبه خطای بهبود یافته
    loss = sum(0.5 * (y - t) ** 2 for y, t in zip(outputs, targets)) / len(outputs)

    # یا می‌توانید MSE استاندارد استفاده کنید:
    def mse_loss(outputs, targets):
        total_loss = 0
        for y, t in zip(outputs, targets):
            total_loss += np.mean((y - t) ** 2)
        return total_loss / len(outputs)

    loss = mse_loss(outputs, targets)

    # Backward pass (BPTT)
    dW_x = np.zeros_like(W_x)
    dW_h = np.zeros_like(W_h)
    dW_y = np.zeros_like(W_y)
    db_h = np.zeros_like(b_h)
    db_y = np.zeros_like(b_y)
    dh_next = np.zeros_like(h_states[0])

    for t in reversed(range(seq_length)):
        dy = outputs[t] - targets[t]
        dW_y += np.dot(dy, h_states[t].T)
        db_y += dy

        dh = np.dot(W_y.T, dy) + dh_next
        dh_raw = dh * dtanh(h_states[t])
        db_h += dh_raw
        dW_x += np.dot(dh_raw, inputs[t].T)
        if t != 0:
            dW_h += np.dot(dh_raw, h_states[t - 1].T)
            dh_next = np.dot(W_h.T, dh_raw)
        else:
            dW_h += np.dot(dh_raw, np.zeros_like(h_states[0]).T)

    # به‌روزرسانی وزن‌ها
    for param, dparam in zip([W_x, W_h, W_y, b_h, b_y], [dW_x, dW_h, dW_y, db_h, db_y]):
        param -= learning_rate * dparam

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {np.sum(loss)}")

Epoch 0, Loss: 0.014341937988029579
Epoch 100, Loss: 0.0021997280741166986
Epoch 200, Loss: 0.001437315055515223
Epoch 300, Loss: 0.0009634108794292478
Epoch 400, Loss: 0.0006604473624690024
Epoch 500, Loss: 0.000460007490018587
Epoch 600, Loss: 0.0003240454545777322
Epoch 700, Loss: 0.00023015256674672796
Epoch 800, Loss: 0.0001644600265232886
Epoch 900, Loss: 0.0001180530004434244


# Learn LSTM

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def tanh(x):
    return np.tanh(x)

class SimpleLSTM:
    def __init__(self, input_size=1, hidden_size=4):
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # اولیه‌سازی وزن‌ها (فرض کنیم از قبل آموزش دیده‌اند)
        np.random.seed(42)
        
        # وزن‌های Forget Gate
        self.Wf = np.random.randn(hidden_size, input_size + hidden_size) * 0.5
        self.bf = np.ones(hidden_size) * 0.1
        
        # وزن‌های Input Gate
        self.Wi = np.random.randn(hidden_size, input_size + hidden_size) * 0.5
        self.bi = np.ones(hidden_size) * 0.1
        
        # وزن‌های Cell State Candidate
        self.Wc = np.random.randn(hidden_size, input_size + hidden_size) * 0.5
        self.bc = np.zeros(hidden_size)
        
        # وزن‌های Output Gate
        self.Wo = np.random.randn(hidden_size, input_size + hidden_size) * 0.5
        self.bo = np.ones(hidden_size) * 0.1
        
        # وزن‌های خروجی نهایی
        self.Wy = np.random.randn(1, hidden_size) * 0.5
        self.by = np.zeros(1)
        
        print("🧠 LSTM آماده شد!")
        print(f"📊 اندازه ورودی: {input_size}, اندازه حافظه مخفی: {hidden_size}")
    
    def forward_step(self, x, h_prev, C_prev, verbose=False):
        """یک گام پیش‌رو در LSTM"""
        
        # ترکیب ورودی فعلی با حافظه قبلی
        combined = np.concatenate([x, h_prev])
        
        # 🚪 Forget Gate - تصمیم گیری برای فراموشی
        f_t = sigmoid(np.dot(self.Wf, combined) + self.bf)
        
        # 🔄 Input Gate - تصمیم گیری برای ورودی جدید
        i_t = sigmoid(np.dot(self.Wi, combined) + self.bi)
        
        # 📝 Cell State Candidate - مقادیر کاندید جدید
        C_tilde = tanh(np.dot(self.Wc, combined) + self.bc)
        
        # 🧠 بروزرسانی Cell State
        C_t = f_t * C_prev + i_t * C_tilde
        
        # 🚪 Output Gate - تصمیم گیری برای خروجی
        o_t = sigmoid(np.dot(self.Wo, combined) + self.bo)
        
        # 📤 Hidden State جدید
        h_t = o_t * tanh(C_t)
        
        # 📈 خروجی نهایی
        y_t = np.dot(self.Wy, h_t) + self.by
        
        if verbose:
            print(f"  🔹 Forget Gate: {f_t.round(3)}")
            print(f"  🔹 Input Gate: {i_t.round(3)}")
            print(f"  🔹 Output Gate: {o_t.round(3)}")
            print(f"  🔹 Cell State: {C_t.round(3)}")
            print(f"  🔹 Hidden State: {h_t.round(3)}")
            print(f"  🔹 پیش‌بینی: {y_t[0]:.3f}")
        
        return y_t, h_t, C_t, (f_t, i_t, o_t, C_tilde)
    
    def predict_sequence(self, input_sequence, verbose=False):
        """پیش‌بینی یک دنباله کامل"""
        
        # اولیه‌سازی حافظه
        h_t = np.zeros(self.hidden_size)
        C_t = np.zeros(self.hidden_size)
        
        predictions = []
        gates_info = []
        
        print(f"🚀 شروع پیش‌بینی دنباله با {len(input_sequence)} عنصر")
        
        for i, x in enumerate(input_sequence):
            if verbose:
                print(f"\n📍 گام {i+1}: ورودی = {x:.3f}")
            
            x_input = np.array([x])
            y_pred, h_t, C_t, gates = self.forward_step(x_input, h_t, C_t, verbose)
            
            predictions.append(y_pred[0])
            gates_info.append(gates)
        
        return np.array(predictions), gates_info

# 📊 تولید دیتای آموزشی - دنباله حسابی ساده
def generate_arithmetic_sequence(start=1, step=2, length=10):
    """تولید دنباله حسابی"""
    return [start + i * step for i in range(length)]

def create_training_data(sequence, window_size=3):
    """تبدیل دنباله به دیتای آموزشی"""
    X, y = [], []
    for i in range(len(sequence) - window_size):
        X.append(sequence[i:i + window_size])
        y.append(sequence[i + window_size])
    return np.array(X), np.array(y)

# 🎯 اجرای مثال
if __name__ == "__main__":
    print("=" * 50)
    print("🔢 مثال LSTM برای پیش‌بینی دنباله حسابی")
    print("=" * 50)
    
    # 1️⃣ تولید دیتا
    print("\n1️⃣ تولید دیتای آموزشی:")
    sequence = generate_arithmetic_sequence(start=1, step=3, length=8)
    print(f"دنباله اصلی: {sequence}")
    
    X_train, y_train = create_training_data(sequence, window_size=3)
    print(f"نمونه آموزشی: {X_train[0]} -> {y_train[0]}")
    
    # 2️⃣ ایجاد مدل
    print("\n2️⃣ ایجاد مدل LSTM:")
    lstm = SimpleLSTM(input_size=1, hidden_size=4)
    
    # 3️⃣ تست پیش‌بینی
    print("\n3️⃣ تست پیش‌بینی با جزئیات:")
    test_sequence = [1, 4, 7]  # بخشی از دنباله برای تست
    print(f"دنباله تست: {test_sequence}")
    
    predictions, gates_info = lstm.predict_sequence(test_sequence, verbose=True)
    
    # 4️⃣ نمایش نتایج
    print("\n4️⃣ خلاصه نتایج:")
    print(f"ورودی‌ها: {test_sequence}")
    print(f"پیش‌بینی‌ها: {predictions.round(3)}")
    
    # پیش‌بینی عنصر بعدی
    next_prediction = predictions[-1]
    expected_next = 10  # عنصر بعدی واقعی در دنباله
    print(f"\n🎯 پیش‌بینی عنصر بعدی: {next_prediction:.3f}")
    print(f"🎯 مقدار واقعی: {expected_next}")
    print(f"🎯 خطا: {abs(next_prediction - expected_next):.3f}")
    
    # 5️⃣ تحلیل گیت‌ها
    print("\n5️⃣ تحلیل عملکرد گیت‌ها:")
    for i, (f_gate, i_gate, o_gate, c_tilde) in enumerate(gates_info):
        print(f"گام {i+1}:")
        print(f"  🔴 میانگین Forget Gate: {f_gate.mean():.3f} (بالا = حفظ حافظه)")
        print(f"  🟢 میانگین Input Gate: {i_gate.mean():.3f} (بالا = پذیرش ورودی)")
        print(f"  🔵 میانگین Output Gate: {o_gate.mean():.3f} (بالا = خروجی فعال)")
    
    print("\n" + "=" * 50)
    print("✅ تمام! حالا می‌تونی ببینی LSTM چطوری کار می‌کنه")
    print("💡 برای یادگیری واقعی، باید وزن‌ها با backpropagation بروزرسانی بشن")
    print("=" * 50)